In [7]:
#Importing Library
import cv2, os, sys
import numpy as np
import time

# Task 1 (Background Estimation):

## Part I:
First create a new image, image_A, wth the same size (No# Px rows and columns) as the input image, which we call image_I.

In [8]:
#Input Image read as Gray scale
image_I = cv2.imread("Particles.png", cv2.IMREAD_GRAYSCALE)
(height, width) = image_I.shape
#Dimension of Input Image
print(f"Input image specifications: Height {height}, Width {width}")
#Create a new image_A by copying image_I
image_A = image_I.copy()
print(f'Image_A created and stored as a variable as specified!')

Input image specifications: Height 320, Width 394
Image_A created and stored as a variable as specified!


## Part II:
Second, the algorithm should go through the pixels of I one by one, and for each pixel(x,y) it must find the maximum gray value in a neighbourhood around that pixel, and write that maximum gray value in the corresponding pixel location (x,y) in A. The resulting image A is called a **max-filtered** image of input image_I. 
<br><br>
The neighbourhood should be of size N X N pixels, where N is a free parameter of the algorithm. Experiement with different values of N and mention in your report what is the smallest value of N that causes the dark particles in I to disappear altogether in image A. Also explain why this value of N causes the particles to disappear.
The max-filtering causes the gray values in A to be higher than the actual background values
in I, so a correction is needed. Extend your algorithm to create another image, which we call
image B here, of the same size as I and A. Now let the algorithm go through the pixels of A
one by one, and for each pixel (x,y) find the minimum gray value in an N x N neighbourhood
around that pixel, and write that minimum gray value in (x,y) in B. The resulting image B is 
called a min-filtered image of the image A.
In your report, include image B computed from Particles.png.

In [9]:
def brute_force_filter_function(input_image, N, indicator):
    '''
    Brute force method, go through matrix via x,y coordinates, and store each pixel intensity value within NxN window 
    into a neighbourhood list, then finally find max or min within the list. 
    :param input_image: numpy image matrix
    :param N: windows size NxN
    :param indicator: max or min filter
    :return: filtered image
    '''
    (height, width) = input_image.shape
    if (N%2) == 0 or N <= 1:
        print("Input Error: max_gray - NxN maxtrix should be an odd number, as even number results in bias choice of anchoring.")
        return 
    #We anchor at center point
    steps = int(N/2)
    #First create a dummy array
    output_image = np.zeros_like(input_image)
    for y in range(0, height):
        for x in range(0, width):
            #We iterate through the x,t coordinates, y represent row number, x represent cell
            #print(f'Current X = {x}, Current Y = {y}')
            neighbourhood = []
            #steps in y
            for j in range(-steps, steps+1):
                #steps in x
                for i in range(-steps, steps+1):
                    if (y+j >= 0 and y+j < height) and (x+i >= 0 and x+i < width):
                        try:
                            neighbourhood.append(input_image[y+j][x+i])
                        except IndexError:
                            continue
            if indicator == 'max':
                output_image[y][x] = max(neighbourhood)
            elif indicator == 'min':
                output_image[y][x] = min(neighbourhood)
    return output_image
def numpy_slicing_filter_function(input_image, N, indicator):
    '''
    numpy slicing filter, go through matrix via x and y coordinates, slice out NxN window by first finding x and y lower 
    and upper bound. once the NxN window is sliced out using numpy amax or amin to find respective value.
    :param input_image: 
    :param N: 
    :param indicator: 
    :return: 
    '''
    (height, width) = input_image.shape
    if (N%2) == 0 or N <= 1:
        print("Input Error: max_gray - NxN maxtrix should be an odd number, as even number results in bias choice of anchoring.")
        return 
    #We anchor at center point
    steps = int(N/2)
    step_list = [i for i in range(-steps, steps+1)]
    reverse_steps_list = step_list[::-1]
    #First create a dummy array
    output_image = np.zeros_like(input_image)
    for y in range(0, height):
        for x in range(0, width):
            #We iterate through the x,t coordinates, y represent row number, x represent cell
            #print(f'Current X = {x}, Current Y = {y}')
            #First Find window Boundaries for x
            #x_lower bound
            for i in step_list:
                if x + i >= 0:
                    x_lower_bound = x+i
                    break
            #y_lower_bound
            for j in step_list:
                if y + j >= 0:
                    y_lower_bound = y+j
                    break
            #x_upper bound
            for i in reverse_steps_list:
                if x + i <= width and x+i >=0:
                    x_upper_bound = x+i
                    break
            #y_upper_bound
            for j in reverse_steps_list:
                if y + j <= height and y+j >=0:
                    y_upper_bound = y+j
                    break
            window_matrix= input_image[y_lower_bound:y_upper_bound+1, x_lower_bound:x_upper_bound+1]
            if indicator == 'max':
                output_image[y][x] = np.amax(window_matrix)
            elif indicator == 'min':
                output_image[y][x] = np.amin(window_matrix)
    return output_image
'''
#To ensure 2 matrix provides same output
for i in range(3, 10, 2):
    image_A_brute = brute_force_filter_function(image_A, i, 'max')
    image_A_numpy = numpy_slicing_filter_function(image_A, i, 'max')
    #Comparing 2 matrix to ensure both outcomes are the same
    filter_function_comparison = image_A_brute == image_A_numpy
    if filter_function_comparison.all() != True:
        print("Output differed!")
print("Completed Comparison test, all output is the same!")
'''
#Change directory to avoid overflooding main directory with image files
main_directory = os.getcwd()
os.chdir('particles/')
'''
#To find the optimal A, we experience with different N value
for i in range(3, 30, 2):
    image_A_filtered = numpy_slicing_filter_function(image_A, i, 'max')
    cv2.imwrite(f'Max_N_{i}.png', image_A_filtered)
'''
#From analysis N = 13 has all particles disappeared, now lets perform min_filter on image_A
image_A = numpy_slicing_filter_function(image_A, 13, 'max')
cv2.imwrite("image_A_selected_N=13.png", image_A)
'''
#Usually N value stays the same for max-min filter, however for the sake of experiement, lets run various N value
#on image_A to see the different effects
for i in range(3, 30, 2):
    image_B_filtered = numpy_slicing_filter_function(image_A, i, 'min')
    cv2.imwrite(f'Min_N_{i}.png', image_B_filtered)
'''
#image_B uses consistent N value:
image_B = numpy_slicing_filter_function(image_A, 13, 'min')
cv2.imwrite("image_B_selected_N=13.png", image_B)
print('Completed! image_A is maxed filtered, image_B is minfilter of image_A, resulting in background of image_I.')

Completed! image_A is maxed filtered, image_B is minfilter of image_A, resulting in background of image_I.


# Task 2 (Background Subtraction):

## Part I:
Now that your algorithm can estimate the background B of an image I, removing the shading artifacts from I can be done by subtracting B pixel by pixel form I, resulting in the output image O. Extend your algorithm to perform this subtraction.

In your report, include image O computed from Particles.png


In [10]:
#O = I - B + 255
#image_B represents the background
#image_I represents the original image
#We need to change the data type to float
image_O = np.subtract(image_I.astype(np.float32), image_B.astype(np.float32)) +255
cv2.imwrite('particles_image_O_raw.png', image_O)
print("Completed! See image_O for output of particles.png under particles directory, filtered and normalised by O = I-B +255.")

Completed! See image_O for output of particles.png under particles directory, filtered and normalised by O = I-B +255.


# Task 3 (Algorithm generalisation):

## Part I
Open the given image **Cells.png** which, similar to **Particles.png**, has a background shading pattern that needs to be removed. There are three main difference between the two images:
- The sizes of the images are different
- The sizes of the objects (cells versus particles) are different
- In **Cells.png** the objects are bright and the background is dark, whereas in Particles.png the objects are dark and the background is bright

Make sure your algorithm can deal with input images of arbitrary size. Dealing with larger objects in images ia a matter of changing the value of the neighbourhood parameter N. But as you will see, your algorithm will not be able to remove the shading from **Cells.png**. To make your algorithm work for that image, you need to reverse the max-filtering and min-filtering.

In [11]:
#Now we need to operate with Cells.png
#First change back to main directory
os.chdir(main_directory)
#load cells.png to image_I, and make a copy
image_I = cv2.imread('Cells.png', cv2.IMREAD_GRAYSCALE)
image_A = image_I.copy()
#Now change to cells directory
os.chdir('cells/')
'''
#Now let's run similar N value test to find optimal N
for i in range(3, 60, 2):
    #run Min-Filter First as Cells image has background dark
    image_A_filtered = numpy_slicing_filter_function(image_A, i, 'min')
    cv2.imwrite(f'Min_N_{i}.png', image_A_filtered)
'''
#From analysis N = 31 has all particles disappeared, now lets perform min_filter on image_A
image_A = numpy_slicing_filter_function(image_A, 31, 'min')
cv2.imwrite("image_A_selected_N=31.png", image_A)
'''
#For analysis lets run image_B with various N
for i in range(3, 60, 2):
    #run Max-Filter on image_A 
    image_B_filtered = numpy_slicing_filter_function(image_A, i, 'max')
    cv2.imwrite(f'Max_N_{i}.png', image_B_filtered)
'''
#Now we use same N for image_B
image_B = numpy_slicing_filter_function(image_A, 31, 'max')
cv2.imwrite("image_B_selected_N=31.png", image_B)
image_O = np.subtract(image_I.astype(np.float32), image_B.astype(np.float32)) 
cv2.imwrite('cells_image_O_raw.png', image_O)
print("Completed! See image_O for output of cells.png under cells directory, filtered and normalised by O = I-B +255.")

Completed! See image_O for output of cells.png under cells directory, filtered and normalised by O = I-B +255.


## Part II
Extend your algorithm with another free parameter, named M. If the user sets M=0, the algorithm should perform max-filtering (image I to A), then min-filtering (image A to B), then subtraction (O = I - B). And if the user sets M = 1, the algorithm should perform first min-filtering, then max-filtering, then subtraction.
 

In [12]:
#Now change back to main directory
os.chdir(main_directory)
#For easier processing, we define another function that calls filter function
def generalised_algorithm(input_image, N, M):
    '''
    generalised_algorithm calls numpy_slicing_filter function, and perform Max-Min or Min-Max filter based on M value,
    :param input_image: Image filename, string with .png
    :param N: Size of N, odd number only
    :param M: 0 for Max then Min Filter, 1 for Min then Max Filter
    :return: None
    '''
    image_I = cv2.imread(input_image, cv2.IMREAD_GRAYSCALE)
    if M == 0: #Max-filter then Min Filter
        #Always create a copy, do not play with original
        image_A = image_I.copy()
        image_A = numpy_slicing_filter_function(image_A, N, 'max')
        image_B = numpy_slicing_filter_function(image_A, N, 'min')
        #Output image
        image_O_raw = np.subtract(image_I.astype(np.float32), image_B.astype(np.float32)) + 255
    elif M == 1: #Min-filter then Max Filter
        #Always create a copy, do not play with original
        image_A = image_I.copy()
        image_A = numpy_slicing_filter_function(image_A, N, 'min')
        image_B = numpy_slicing_filter_function(image_A, N, 'max')    
        #Output image
        image_O_raw = np.subtract(image_I.astype(np.float32), image_B.astype(np.float32))
    #Perform Contrast Stretching
    a,b = 0,255
    c,d = np.amin(image_O_raw), np.amax(image_O_raw)
    image_O = (image_O_raw - c) * ((b-a)/(d-c)) + a
    image_O = np.uint8(image_O)
    show_image = np.concatenate((image_I, image_A), axis=1)
    show_image = np.concatenate((show_image, image_B), axis=1)
    show_image = np.concatenate((show_image, image_O_raw), axis=1)
    show_image = np.concatenate((show_image, image_O), axis=1)
    cv2.imwrite("image_O_raw.png", image_O_raw)
    cv2.imwrite("contrast_stretched_image_O.png", image_O)
    cv2.imwrite("input_image_filtered_process.png", show_image)
    #Add any filter function here and then concatenate it to a single image
    return 
start_time = time.time()
generalised_algorithm("Cells.png", 31, 1) #To try different image, input here
print(f"Completed! Total process time is {time.time() - start_time}")

Completed! Total process time is 9.35074234008789
